<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Chap_02_RNN_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse

import string
import random
import re
import time, math

In [2]:
num_epochs = 1000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

In [3]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)
print(n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100
100


In [4]:
import tensorflow as tf
data_dir = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',cache_subdir='/content/sample_data')  # shakespeare
# 학습에 사용할 txt 파일을 읽습니다.
file = open(data_dir, 'rb').read().decode(encoding='utf-8')
file_len = len(file)
print('file_len =', file_len)

1115394/1115394 [==============================] - 0s 0us/step
file_len = 1115394


In [5]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

S:
O, not by much.

PAULINA:
So much the more our carver's excellence;
Which lets go by some sixteen years and makes her
As she lived now.

LEONTES:
As now she might have done,
So much to my good comfo


In [6]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)#.cuda()

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [7]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [8]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden,cell):
        out = self.encoder(input.view(batch_size,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden,cell

    def init_hidden(self):
          
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        
        return hidden,cell
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters-1, num_layers)#.cuda()

In [9]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 99])


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()


In [11]:
def test(seq):
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(seq):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [12]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden,cell = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        target = label[j]
        y,hidden,cell = model(x,hidden,cell)
        y = y.squeeze()
        target = target.squeeze()
        loss += loss_func(y,target)
    loss.backward()
    optimizer.step()
  # 5회 반복마다 파라미터를 checkpoint로 저장합니다.
    if (i) % 50 == 0:
      print ('------------------------------------------------------')
      print ('Epoch {} Loss {:.4f}'.format(i+1, loss))
      print("샘플링을 시작합니다!")
      print ('------------------------------------------------------')
      test(200)
      print('\n')
      print ('------------------------------------------------------')
      print("샘플링이 끝났습니다.!")



------------------------------------------------------
Epoch 1 Loss 918.1113
샘플링을 시작합니다!
------------------------------------------------------
b0*&aM[)=Zo?@=~7qhtV? Ku/fp,Qi17FBq1Ol864i@pAn;@GO)"06$yWf^$mMU
2X}k5t|?n;3	"OHc]CtT5Or\D&\*lB^cUy;
|~C%8Y!U(+s3-^Rzq#w([	;Sj+FJe_#xVZGMB0=?eo,P{Apc344=[

------------------------------------------------------
샘플링이 끝났습니다.!
------------------------------------------------------
Epoch 51 Loss 548.5011
샘플링을 시작합니다!
------------------------------------------------------
be the t liranaonorec ihes wesve Yhe betdl noul hou po sins fouse monucsas then wandse,
e reorerp;s ear the tha2p sto f@otBe tcelan anuhaO.
T
-the d ote imive theethe: we aute oar so aano and ous wanel

------------------------------------------------------
샘플링이 끝났습니다.!
------------------------------------------------------
Epoch 101 Loss 471.6416
샘플링을 시작합니다!
------------------------------------------------------
bhe fis roo pniandes hint shert sthe ove sat ie.
I fog kin nthe sone t

In [13]:
print ('------------------------------------------------------')
print("최종 샘플링을 시작합니다!")
print ('------------------------------------------------------')
test(2000)
print('\n')
print ('------------------------------------------------------')
print("최종 샘플링이 끝났습니다.!")

------------------------------------------------------
최종 샘플링을 시작합니다!
------------------------------------------------------
by
Fy say preanty leker,
To all hard, I my and sent to lind bece words, with it thearts mered them. fors of you a say.

LARDI:
Gole to love harat the to to lord ink,
To thou shave killo, let to till, the buse por is shand not beake the s a spearsice.

FRISHAR:
And of Our aild thed her fore, Ediveringers for hing-thers will to cangere hert, and the man not to law you the whan more hear in to thel, wray, friol---sonts for my so thear freredour band,
Hee mo shill know to a more come, mean enthly horse.

II ETRO:
The, a mantle womet sup but renting for on the priting to lorking for had demans. hear you me mordered eain?

METINCENTE:
As for and to ir not ent this ambut bouror you of home:
I and gord mon whis ceroth you come tamer geit stare gowl! I for that jure thous maince did the rak ming this seal in this day there gomes so woard where me, and that sle the comnold